In [2]:
# imports of python libraries
import requests
import os
import pandas as pd
import json

In [15]:
# import citybike dataframe 
import pandas as pd
citybike_df= pd.read_csv(
                        f"/Users/tinapham/Desktop/lighthouse-data-notes/ProJect/LHL_StatsModel_PythonProject/data/citybike.csv", 
                        sep=","
                        )
citybike_df

,Station_name,Available_bikes,Longitude,Latitude
0,Cégep Marie-Victorin,10,-73.606011,45.617500
1,Gare d'autocars de Montréal (Berri / Ontario),6,-73.564257,45.516926
2,Molson / William-Tremblay,6,-73.565012,45.541549
3,Ateliers municipaux de St-Laurent (Cavendish /...,11,-73.711186,45.506176
4,Place Rodolphe-Rousseau (Gohier / Édouard-Laurin),7,-73.682498,45.512994
...,...,...,...,...
757,Jean-Talon / de l'Épée,14,-73.624063,45.527487
758,Gare LaSalle (Highlands),4,-73.657136,45.425613
759,Métro St-Michel (Shaughnessy / St-Michel),2,-73.599488,45.559076
760,St-Hubert / René-Levesque,4,-73.556487,45.514369


In [20]:
# Set API KEY with user's environment variables
import os
FOUSQUARE_API_KEY   =os.environ["FOURSQUARE_API_KEY2"]
YELP_API_KEY        =os.environ["YELP_API_KEY"]

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [21]:
import requests
# NOTE: API get request only accept latitude and longitude as a string
# Create a list of latitudes and longitudes from the citybike dataframe to pass through a for loop
latitudes = citybike_df['Latitude'].tolist()
longitudes = citybike_df['Longitude'].tolist()

# Create an empty dictionary to store the json object from the API requist
list_4square0=[]

# Create a for loop that iterates over the list of (latitudes, longitudes) which can then be use to loop into a request url 
for latitude, longitude in zip(latitudes, longitudes):
    
    # Create a Foursquare API request within the for loop to iterates response over each pair of (latitudes, longitudes)
    resquest_4square0 = requests.get(
        "https://api.foursquare.com/v3/places/search",
        params={
            "query": ["bar","restaurant","park","museum","market","grocery"],
            "ll": f"{latitude},{longitude}",
            "radius": 1000,
            "limit": 50,
        },
        headers= {
                "Accept": "application/json",
                "Authorization": FOUSQUARE_API_KEY
                }
    )
    list_4square0.append(resquest_4square0.json())

print(resquest_4square0)
    

<Response [429]>


In [22]:
# QA: number of key in the 4square response dictionary should match with number of row in citybike
response_dict=len(list_4square0)
city_bike_df= citybike_df['Longitude'].count()
if response_dict==city_bike_df:
    print('Match')
else:
    print('Fail')

Match


In [ ]:
import json
# Save response as .json file to limit amount of API request
# with open("response_4square_list0.json", "w") as outfile:
    #json.dump(list_json_4square0, outfile)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [25]:
with open ('/Users/tinapham/Desktop/lighthouse-data-notes/Week_5/Project draft/response_4square3_raw.json', 'r') as f:
     list_4square0=json.load(f)

list_4square_raw= list_4square0[0:762]
list_4square_raw[0]

{'results': [{'fsq_id': 'ec53d9eccf6a445dc5ad4526',
   'categories': [{'id': 13022,
     'name': 'Sports Bar',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/sportsbar_',
      'suffix': '.png'}},
    {'id': 13236,
     'name': 'Italian Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 246,
   'geocodes': {'drop_off': {'latitude': 45.61913, 'longitude': -73.608237},
    'main': {'latitude': 45.618905, 'longitude': -73.608463},
    'roof': {'latitude': 45.618905, 'longitude': -73.608463}},
   'link': '/v3/places/ec53d9eccf6a445dc5ad4526',
   'location': {'address': '7000 Maurice-Duplessis Blvd',
    'country': 'CA',
    'formatted_address': '7000 Maurice-Duplessis Blvd, Montreal QC',
    'locality': 'Montreal',
    'region': 'QC'},
   'name': 'Resto-bar Capucine - Nord-Est de Montréal',
   'related_places': {}},
  {'fsq_id': '510ac01270430a0e5851f534',
   'cat

In [24]:
# Testing Obtain POI Name
list_4square0[0]['results'][0]['name']
# Json structure list--> result--> list-->'name'

'Resto-bar Capucine - Nord-Est de Montréal'

In [26]:
# Empty list to contain all iterate object from the for loop
list_4squareb=[]

# Create the first for loop to extract data from results
for i in range(len(list_4square_raw)):
    list_4square_a=(list_4square0[i]['results'])
    
    # Create the second for loop to get data inside the list
    for n in range(len(list_4square_a)):
        list_4squareb.append(list_4square_a[n])

# Parse list into a dataframe
raw_4square_df=pd.json_normalize( list_4squareb)

# Drop duplicate row from 'fsq_id'
df_4square= raw_4square_df.drop_duplicates(subset='fsq_id')

In [27]:
df_4square

,fsq_id,categories,chains,distance,link,name,geocodes.drop_off.latitude,geocodes.drop_off.longitude,geocodes.main.latitude,geocodes.main.longitude,...,location.formatted_address,location.locality,location.region,timezone,location.cross_street,location.postcode,related_places.children,location.address_extended,related_places.parent.fsq_id,related_places.parent.name
0,ec53d9eccf6a445dc5ad4526,"[{'id': 13022, 'name': 'Sports Bar', 'icon': {...",[],246,/v3/places/ec53d9eccf6a445dc5ad4526,Resto-bar Capucine - Nord-Est de Montréal,45.619130,-73.608237,45.618905,-73.608463,...,"7000 Maurice-Duplessis Blvd, Montreal QC",Montreal,QC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,510ac01270430a0e5851f534,"[{'id': 17082, 'name': 'Furniture and Home Sto...",[],874,/v3/places/510ac01270430a0e5851f534,Barazin | manufacturier portes aluminium et ve...,45.610071,-73.609681,45.609896,-73.609624,...,6440 Henri-Bourassa Blvd E (Boulevard Langelie...,Montréal,QC,America/Toronto,Boulevard Langelier,H1G 5W9,NaN,NaN,NaN,NaN
2,4c3fa4d33735be9a4bbc15a4,"[{'id': 13022, 'name': 'Sports Bar', 'icon': {...",[],1107,/v3/places/4c3fa4d33735be9a4bbc15a4,Boca Bar & Grill,NaN,NaN,45.624006,-73.601007,...,"7310 Maurice-Duplessis Blvd, Montreal QC H1E 1M4",Montréal-Nord,QC,America/Toronto,NaN,H1E 1M4,NaN,NaN,NaN,NaN
3,598f0623747e41848ab1057c,"[{'id': 13009, 'name': 'Cocktail Bar', 'icon':...",[],661,/v3/places/598f0623747e41848ab1057c,Piano Bar la Belle Epoque,NaN,NaN,45.611645,-73.607545,...,"6711 Henri Bourassa Blvd Est N, Montreal-Nord ...",Montreal-Nord,QC,NaN,NaN,H1G 2V6,NaN,NaN,NaN,NaN
4,27709dd5a7964399a5be91be,"[{'id': 11153, 'name': 'Recycling Facility', '...",[],669,/v3/places/27709dd5a7964399a5be91be,Barils International Ltée,NaN,NaN,45.617824,-73.597406,...,"11390 4e Ave, Montreal QC H1E 3A6",Montreal,QC,NaN,NaN,H1E 3A6,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23112,dafc2b8162794c029eec728a,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],909,/v3/places/dafc2b8162794c029eec728a,Bar Sarah B,NaN,NaN,45.502798,-73.560492,...,"360 Saint-Antoine Rue O, Montreal QC H2Y 3X4",Montreal,QC,NaN,NaN,H2Y 3X4,NaN,NaN,NaN,NaN
23113,27ada6bbc189450224f8aaf9,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],914,/v3/places/27ada6bbc189450224f8aaf9,Friscos Deli-Bar & Grill,NaN,NaN,45.502557,-73.561162,...,"405 Saint-Antoine Rue O, Montreal QC H2Z 2A3",Montreal,QC,NaN,NaN,H2Z 2A3,NaN,NaN,NaN,NaN
23114,8d0c447cf9324b2934dbe1c9,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],925,/v3/places/8d0c447cf9324b2934dbe1c9,Restaurant Bar Delmo,45.503682,-73.557602,45.503763,-73.557713,...,"211 Notre-Dame Rue W, Montreal QC H2Y 1T4",Montreal,QC,NaN,NaN,H2Y 1T4,NaN,NaN,NaN,NaN
23116,e69247c709224afc4b9bf9a7,"[{'id': 13065, 'name': 'Restaurant', 'icon': {...",[],939,/v3/places/e69247c709224afc4b9bf9a7,Biddle's Jr. Resto Cabaret Bar,45.503707,-73.557584,45.503874,-73.557814,...,"209 Notre Dame Rue O, Montreal QC",Montreal,QC,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Put your parsed results into a DataFrame

In [29]:
# Create an empty dictionary with key as the column for dataframe
Clean_4square_dict={"POI_Name":[],
                "POI_distance":[]}

# Insert data into each column
Clean_4square_dict['POI_Name']=df_4square['name']
Clean_4square_dict['POI_distance']=df_4square['distance'].astype(int)
Clean_4square_dict['POI_categories']= df_4square['categories']
Clean_4square_dict['POI_Latitude']=df_4square['geocodes.main.latitude'].astype(float)
Clean_4square_dict['POI_Longitude']=df_4square['geocodes.main.longitude'].astype(float)

# Turn ditionary into a dataframe
Clean_4square_df= pd.DataFrame(Clean_4square_dict)
Clean_4square_df


,POI_Name,POI_distance,POI_categories,POI_Latitude,POI_Longitude
0,Resto-bar Capucine - Nord-Est de Montréal,246,"[{'id': 13022, 'name': 'Sports Bar', 'icon': {...",45.618905,-73.608463
1,Barazin | manufacturier portes aluminium et ve...,874,"[{'id': 17082, 'name': 'Furniture and Home Sto...",45.609896,-73.609624
2,Boca Bar & Grill,1107,"[{'id': 13022, 'name': 'Sports Bar', 'icon': {...",45.624006,-73.601007
3,Piano Bar la Belle Epoque,661,"[{'id': 13009, 'name': 'Cocktail Bar', 'icon':...",45.611645,-73.607545
4,Barils International Ltée,669,"[{'id': 11153, 'name': 'Recycling Facility', '...",45.617824,-73.597406
...,...,...,...,...,...
23112,Bar Sarah B,909,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",45.502798,-73.560492
23113,Friscos Deli-Bar & Grill,914,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",45.502557,-73.561162
23114,Restaurant Bar Delmo,925,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",45.503763,-73.557713
23116,Biddle's Jr. Resto Cabaret Bar,939,"[{'id': 13065, 'name': 'Restaurant', 'icon': {...",45.503874,-73.557814


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [18]:
import requests
import json
list_yelp0=[]
# NOTE: Yelp limit to 500 call per day, use limit to break the API call
index= 0
limit= 10
for item, series in citybike_df.iterrows():
    lat = series['Latitude']
    long = series ['Longitude']

    url_yelp = "https://api.yelp.com/v3/businesses/search"
    params= {"latitude": lat,
            "longitude": long,
            "categories": ["localflavor",
                           "landmark",
                           "park", 
                           "restaurant",
                           "bar"],
            "radius": 1000,
            "limit": 50}
    headers = {
            "accept": "application/json",
            "Authorization": 'Bearer %s' % YELP_API_KEY
            }
    response_yelp = requests.get(url_yelp, headers=headers, params=params)
    data_yelp=response_yelp.json()
    index += 1
    list_yelp0.append(data_yelp)
    if index == limit:
        break

print(response_yelp)

<Response [200]>


In [ ]:
# Save data to limit API call
# with open("response_yelp_list_raw.json", "w") as outfile:
    #json.dump(list_yelp0, outfile)

In [19]:
list_yelp0

[{'businesses': [],
  'total': 0,
  'region': {'center': {'longitude': -73.60601127147675,
    'latitude': 45.617499783128075}}},
 {'businesses': [{'id': 'ioUzNer0HpiFZ1KZ1ez4Pg',
    'alias': 'vieux-montréal-montréal-2',
    'name': 'Vieux-Montréal',
    'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/FUeVotm5YUultpLvCKFnqg/o.jpg',
    'is_closed': False,
    'url': 'https://www.yelp.com/biz/vieux-montr%C3%A9al-montr%C3%A9al-2?adjust_creative=RiQRXOkjsGiSaBL1W4QhpQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=RiQRXOkjsGiSaBL1W4QhpQ',
    'review_count': 85,
    'categories': [{'alias': 'localflavor', 'title': 'Local Flavor'}],
    'rating': 4.5,
    'coordinates': {'latitude': 45.5091617910897,
     'longitude': -73.5534195229411},
    'transactions': [],
    'location': {'address1': '',
     'address2': '',
     'address3': '',
     'city': 'Montreal',
     'zip_code': 'H2X 1X6',
     'country': 'CA',
     'state': 'QC',
     'display_address': ['Montre

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [4]:
# Due to limited acess to Yelp, result are save 
with open ('/Users/tinapham/Desktop/lighthouse-data-notes/Week_5/Project draft/response_yelp_list_raw.json', 'r') as f:
     list_yelp0=json.load(f)

# Create the list to filter out error call message
list_yelp_raw=list_yelp0[0:500]

In [6]:
# Create list of data to be included in the dataframe
Yelp_id=[]
Yelp_name=[]
Yelp_latitude=[]
Yelp_longitude=[]
Yelp_rating=[]
Yelp_review_count=[]
station_distance=[]
station_latitude= []
station_longitude=[]
# Create the first for loop to get data inside the list
for i in range(len(list_yelp_raw)):
    list_yelp_a= list_yelp_raw[i]['businesses']
    # Create the second for loop to get data inside the list
    for n in range(len(list_yelp_a)):
        station_latitude.append(list_yelp_raw[i]['region']['center']['latitude'])
        station_longitude.append(list_yelp_raw[i]['region']['center']['longitude'])
        Yelp_id.append(list_yelp_a[n]['id'])
        Yelp_name.append(list_yelp_a[n]['name'])
        Yelp_latitude.append(list_yelp_a[n]['coordinates']['latitude'])
        Yelp_longitude.append(list_yelp_a[n]['coordinates']['longitude'])
        station_distance.append(list_yelp_a[n]['distance'])
        Yelp_rating.append(list_yelp_a[n]['rating'])
        Yelp_review_count.append(list_yelp_a[n]['review_count'])  

In [9]:
# Create Dict to contain all the info
Yelp_data={'yelp_id':Yelp_id,
            'yelp_name':Yelp_name,
            'yelp_latitude':Yelp_latitude,
            'yelp_longitude':Yelp_longitude,
            'yelp_rating': Yelp_rating,
            'yelp_review_count': Yelp_review_count,
            'station_distance': station_distance,
            'station_latitude':station_latitude,
            'station_longitude':station_longitude
        }


Put your parsed results into a DataFrame

In [8]:
# Turn dictionary into a dataframe
Yelp_df=pd.DataFrame(Yelp_data)
Yelp_df

,yelp_id,yelp_name,yelp_latitude,yelp_longitude,yelp_rating,yelp_review_count,station_distance,station_latitude,station_longitude
0,ioUzNer0HpiFZ1KZ1ez4Pg,Vieux-Montréal,45.509162,-73.553420,4.5,85,1207.697762,45.516926,-73.564257
1,WeWHgusw0QYnO-BCIz5UCw,Quartier des Spectacles,45.512570,-73.563706,5.0,11,486.289044,45.516926,-73.564257
2,SpYlUladVGPg-YAQTliMCg,Quartier Latin,45.516247,-73.566078,4.5,4,189.464763,45.516926,-73.564257
3,LO4yy547h_6ocwaWiiQulw,Juste pour Rire,45.512195,-73.568407,4.5,16,617.477091,45.516926,-73.564257
4,bBGnKJ5e65UIWDppyEJbcA,Mural,45.513580,-73.571370,4.5,9,670.736174,45.516926,-73.564257
...,...,...,...,...,...,...,...,...,...
1653,kElrLZ73R5ednlEghDzaSg,Centre sportif de Notre-Dame-de-Grâce,45.466390,-73.632020,3.5,3,1237.485926,45.459761,-73.644775
1654,KxkEA3Yiws79PHXYivNL5w,Petite Italie,45.533639,-73.610434,4.0,7,1034.452167,45.539891,-73.620270
1655,ixwaNfjo7FsZj7XeycPuIw,Place Shamrock,45.534960,-73.616015,3.5,2,640.648603,45.539891,-73.620270
1656,KxkEA3Yiws79PHXYivNL5w,Petite Italie,45.533639,-73.610434,4.0,7,751.819367,45.535709,-73.619623


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp provided more complete data. Yelp provided distance from each bike station, rating for POI, review count and details about each business. Foursquare does has rating on their normal website, however, maybe it is not provided due to data integrity.

Get the top 10 restaurants according to their rating

In [12]:
# filter out duplicate location from Yelp
Yelp=Yelp_df.drop_duplicates(subset='yelp_id')

In [14]:
# top 10 places with the most review
Yelp.sort_values(by="yelp_review_count", ascending=False).head(10)

,yelp_id,yelp_name,yelp_latitude,yelp_longitude,yelp_rating,yelp_review_count,station_distance,station_latitude,station_longitude
0,ioUzNer0HpiFZ1KZ1ez4Pg,Vieux-Montréal,45.509162,-73.553420,4.5,85,1207.697762,45.516926,-73.564257
28,eI6KvWEe_IJUGawBNH7qXA,Ville souterraine RESO /Underground City,45.500301,-73.565495,3.5,59,1110.331085,45.507144,-73.555119
24,u8wCRXaZNrdanRP0FC-fLw,La Grande Roue de Montréal,45.508719,-73.548957,4.0,47,493.231201,45.507144,-73.555119
53,BZ7PexoOG5x4YQMUf90FqA,Rue Sainte-Catherine,45.502974,-73.569925,3.5,40,1048.361818,45.497165,-73.559330
90,X1T5J9B823N2WK9p4r6E1g,Festival International de Jazz,45.506350,-73.569090,4.5,36,498.987304,45.507838,-73.563136
232,54_git0HB9xRH9o4FCPFfQ,Tam Tam Mont Royal,45.514287,-73.585653,4.5,34,395.896698,45.516897,-73.589108
21,33Pq0cQn3jODiIG7VwUxhw,Yelp Booth,45.506278,-73.552028,4.5,32,259.432827,45.507144,-73.555119
25,yZYXFk0cMBdFVuFd6FC8pA,Ville de Montréal,45.500484,-73.568359,4.5,31,1167.096663,45.507144,-73.555119
6,u95lfFlmAHOlXCs-Whmt9g,Quartier Chinois / Chinatown,45.507938,-73.560022,3.5,26,1052.519087,45.516926,-73.564257
327,R_nf8ZVc_TreNyMN_vLMjA,Sports de Combats,45.526303,-73.595251,5.0,18,624.518108,45.529668,-73.601669


#### Joining data to citybike_df

In [30]:
# Create a for loop to count for POI belong to each bike station from Foursquare
POI_4square=[]
# Parsing data from results
for i in list_4square0:
    POI_4square.append(i['results'])
# Count for children from the parents results
Count_4square_POI=[]
for n in range(len(POI_4square)):
    Count_4square_POI.append(len(POI_4square[n]))
   

In [60]:
# The second call of yelp API for the last 262 stations
with open ('/Users/tinapham/Desktop/lighthouse-data-notes/Week_5/Project draft/response_yelp_list_raw3.json', 'r') as f:
     list_yelp1=json.load(f)

list_yelp1=list_yelp1[0:262]

In [96]:
POI_Yelp_1=[]
Count_Yelp_POI_1=[]

In [97]:
# Getting Yelp POI count
# Parsing json object from businesses
for i in range(len(list_yelp_raw)):
    POI_Yelp_1.append(list_yelp_raw[i]['businesses'])

In [98]:
len(POI_Yelp_1)

500

In [99]:
Count_Yelp_POI=[]
# Add the data second request to the first API request
for i in range(len(list_yelp1)):
    POI_Yelp_1.append(list_yelp1[i]['businesses'])
    # Count for children from the parents results


In [102]:
# QA check if addition from the second API call is properly added to the list
len(POI_Yelp_1)

762

In [100]:
Count_Yelp_POI=[]
# Count for children from the parents results
for n in range(len(POI_Yelp_1)):
    Count_Yelp_POI.append(len(POI_Yelp_1[n]))

In [103]:
# QA check if the number of count match bike station
len(Count_Yelp_POI)

762

In [104]:
# Joining the data to citybike_df
citybike_df["Count_4square_POI"]=Count_4square_POI
citybike_df["Count_Yelp_POI"]=Count_Yelp_POI

In [105]:
citybike_df

,Station_name,Available_bikes,Longitude,Latitude,Count_4square_POI,Count_Yelp_POI
0,Cégep Marie-Victorin,10,-73.606011,45.617500,11,0
1,Gare d'autocars de Montréal (Berri / Ontario),6,-73.564257,45.516926,50,14
2,Molson / William-Tremblay,6,-73.565012,45.541549,20,0
3,Ateliers municipaux de St-Laurent (Cavendish /...,11,-73.711186,45.506176,0,0
4,Place Rodolphe-Rousseau (Gohier / Édouard-Laurin),7,-73.682498,45.512994,22,0
...,...,...,...,...,...,...
757,Jean-Talon / de l'Épée,14,-73.624063,45.527487,43,0
758,Gare LaSalle (Highlands),4,-73.657136,45.425613,2,0
759,Métro St-Michel (Shaughnessy / St-Michel),2,-73.599488,45.559076,24,0
760,St-Hubert / René-Levesque,4,-73.556487,45.514369,50,0


In [ ]:
# Save the dataframe for later usage
#citybike_df.to_csv(r'/Users/tinapham/Desktop/lighthouse-data-notes/ProJect/LHL_StatsModel_PythonProject/data/joindf_clean.cvs', index=False)